In [2]:
import mido
import time
import numpy as np

def normalize(P):
    K = P.shape[0]
    norm = np.sum(P, axis=0, keepdims=False)
    return P / norm

# input the file name
midi_string = 'sample2'
midi_file = mido.MidiFile('%s.mid' % midi_string)

# play audio
play_audio = False

current_tempo = 0
time_signature_numerator = 0
time_signature_denominator = 0
time_signature_quarter = 0
ticks_per_beat = midi_file.ticks_per_beat
print('ticks per beat: %d' % ticks_per_beat)

# bar_duration: reset every bar (unit: ticks)
# total_duration: current cummulative duration (unit: seconds)
bar_duration = 0
total_duration = 0

# notes_vec: store the bar duration of the 128 notes in each 16 channels
# notes_vec_on: 'True' is note on, 'False' is note off (or velocity = 0)
notes_vec = np.zeros([16, 128], dtype=np.float16)
notes_vec_on = np.zeros([16, 128], dtype=np.bool)

# use 'print(mido.get_output_names())' to get your port's name and type it in the 'mido.open_output' bracket
port = None
port = mido.open_output('Microsoft GS Wavetable Synth 0')

# turn numeric notes into english notes, octave info not preserved
def note_int2char(note):
    return {
        '0': 'C',
        '1': 'Db',
        '2': 'D',
        '3': 'Eb',
        '4': 'E',
        '5': 'F',
        '6': 'F#',
        '7': 'G',
        '8': 'Ab',
        '9': 'A',
        '10': 'Bb',
        '11': 'B'
    }.get(str(note % 12), 'undef')

# define the chords and its relative position
chords = np.zeros([24, 12], dtype=np.int8)
chords = np.array(
    [
        # Root note only
        [4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4],
        # Power chords
        [4, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, -1, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, -1, -1, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1, -1],
        [-1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, 4],
        # Major chords
        [4, -1, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, -1, 2, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, -1, 2, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, -1, 2, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, -1, 2, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, -1, 2, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, -1, 2, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1, 2],
        [2, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, -1],
        [-1, 2, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, -1, 2, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, -1, 2, -1, -1, 2, -1, -1, -1, -1, 4],
        # Minor chords
        [4, -1, -1, 2, -1, -1, -1, 2, -1, -1, -1, -1],
        [-1, 4, -1, -1, 2, -1, -1, -1, 2, -1, -1, -1],
        [-1, -1, 4, -1, -1, 2, -1, -1, -1, 2, -1, -1],
        [-1, -1, -1, 4, -1, -1, 2, -1, -1, -1, 2, -1],
        [-1, -1, -1, -1, 4, -1, -1, 2, -1, -1, -1, 2],
        [2, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1, -1],
        [-1, 2, -1, -1, -1, -1, 4, -1, -1, 2, -1, -1],
        [-1, -1, 2, -1, -1, -1, -1, 4, -1, -1, 2, -1],
        [-1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1, 2],
        [2, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1, -1],
        [-1, 2, -1, -1, -1, 2, -1, -1, -1, -1, 4, -1],
        [-1, -1, 2, -1, -1, -1, 2, -1, -1, -1, -1, 4],
    ])

# name the chord
def name_chord(chord):
    return {
        '0': 'C1',
        '1': 'Db1',
        '2': 'D1',
        '3': 'Eb1',
        '4': 'E1',
        '5': 'F1',
        '6': 'F#1',
        '7': 'G1',
        '8': 'Ab1',
        '9': 'A1',
        '10': 'Bb1',
        '11': 'B1',
        '12': 'C5',
        '13': 'Db5',
        '14': 'D5',
        '15': 'Eb5',
        '16': 'E5',
        '17': 'F5',
        '18': 'F#5',
        '19': 'G5',
        '20': 'Ab5',
        '21': 'A5',
        '22': 'Bb5',
        '23': 'B5',
        '24': 'C',
        '25': 'Db',
        '26': 'D',
        '27': 'Eb',
        '28': 'E',
        '29': 'F',
        '30': 'F#',
        '31': 'G',
        '32': 'Ab',
        '33': 'A',
        '34': 'Bb',
        '35': 'B',
        '36': 'Cm',
        '37': 'Dbm',
        '38': 'Dm',
        '39': 'Ebm',
        '40': 'Em',
        '41': 'Fm',
        '42': 'F#m',
        '43': 'Gm',
        '44': 'Abm',
        '45': 'Am',
        '46': 'Bbm',
        '47': 'Bm',
    }.get(str(chord), 'undef')

try:

    for msg in midi_file:
        
        # delay for playing
        if play_audio:
            time.sleep(msg.time)

        if msg.time != 0:

            total_duration += msg.time
            bar_duration += mido.second2tick(msg.time, ticks_per_beat, current_tempo)

            # indication of 'bar_duration' reaching the duration of a complete bar
            if bar_duration / ticks_per_beat >= time_signature_quarter:

                # break the continue-over-bars note-on notes
                notes_vec[np.where(notes_vec_on)] += total_duration
                
                # reset 'bar_duration'
                bar_duration_prev = bar_duration
                bar_duration = bar_duration - ticks_per_beat * time_signature_quarter * (bar_duration // (ticks_per_beat * time_signature_quarter))
                
                # find out the played notes of the complete bar
                played_notes_idx = np.where(notes_vec != 0)
                if played_notes_idx[0].size > 0:
                    played_notes = np.zeros(12, dtype=np.float16)
                    for played_note_idx in np.nditer(played_notes_idx):
                        played_notes[played_note_idx[1] % 12] += notes_vec[played_note_idx[0]][played_note_idx[1]]
                    played_notes = normalize(played_notes)
                                        
                    # give a better 'print' for played notes (all values are from 'played_notes')
                    played_notes_print = [np.where(played_notes != 0)[0], played_notes[played_notes != 0]]
                    played_notes_printf = ''
                    for played_note_print in np.nditer(played_notes_print):
                        played_notes_printf += '{:<2}'.format(note_int2char(played_note_print[0])) + ': ' + "{0:.2f}".format(played_note_print[1]) + '  '
                    
                    # calculate the chord similarity score
                    chords_score = np.dot(chords, played_notes)
                    
                    # print the chord analysis result
                    print('%s\t%s' % (name_chord(np.argmax(chords_score)), played_notes_printf))
                        
                # reset the notes_vec and notes_vec_on for next bar
                notes_vec = np.zeros([16, 128], dtype=np.float16)
                notes_vec_on = np.zeros([16, 128], dtype=np.bool)
                
                # continue the continue-over-bars-note-on notes
                notes_vec[np.where(notes_vec_on)] -= total_duration
                
                # debug message for bar detection
                # print('next bar:\t%.2f to %.2f' % (bar_duration_prev, bar_duration))

        if msg.is_meta:

            # change time signature should be the first beat
            if msg.type == 'time_signature':
                time_signature_denominator = msg.denominator
                time_signature_numerator = msg.numerator
                time_signature_quarter = time_signature_numerator / (time_signature_denominator / 4)
                bar_duration = 0
                print('time signature:\t%d/%d (quarters: %d)' % (time_signature_numerator, time_signature_denominator, time_signature_quarter))

            # update tempo for 'second2tick'
            if msg.type == 'set_tempo':
                current_tempo = msg.tempo
                print("set tempo:\t%d (bpm: %.2f)" % (msg.tempo, mido.tempo2bpm(msg.tempo)))

        if not msg.is_meta:

            # play the note
            if play_audio:
                port.send(msg)
            
            if msg.type == 'note_on':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
            
                    if msg.velocity != 0:
                        if not notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] -= total_duration
                            notes_vec_on[msg.channel][msg.note] = True
                    else:
                        if notes_vec_on[msg.channel][msg.note]:
                            notes_vec[msg.channel][msg.note] += total_duration
                            notes_vec_on[msg.channel][msg.note] = False

            if msg.type == 'note_off':
                
                # ignore drum channel (midi channel = 10)
                if msg.channel != 9:
                    
                    if notes_vec_on[msg.channel][msg.note]:
                        notes_vec[msg.channel][msg.note] += total_duration
                        notes_vec_on[msg.channel][msg.note] = False

        # you can set auto stop in here (seconds as unit)
        if total_duration == -1:

            if port is not None:
                port.close()
            break

    if port is not None:
        port.close()
    print('End of midi')
        
# action after pressing stop
except KeyboardInterrupt:
    
    if port is not None:
        port.close()
    print('Forced stop')

ticks per beat: 480
time signature:	4/4 (quarters: 4)
set tempo:	517242 (bpm: 116.00)
Gm	D : 0.08  F : 0.08  G : 0.42  Bb: 0.42  
Eb	C : 0.18  Eb: 0.18  F : 0.09  G : 0.36  Bb: 0.18  
D	C : 0.18  D : 0.28  Eb: 0.18  F : 0.04  F#: 0.14  A : 0.18  
Bb	C : 0.18  D : 0.18  F : 0.18  G : 0.09  Ab: 0.09  Bb: 0.27  
Gm	D : 0.50  Eb: 0.17  G : 0.17  Bb: 0.17  
Ebm	Db: 0.21  Eb: 0.45  F#: 0.17  Bb: 0.17  
set tempo:	451127 (bpm: 133.00)
Am	C : 0.23  E : 0.23  G : 0.04  A : 0.23  B : 0.27  
Am	C : 0.09  E : 0.09  G : 0.16  A : 0.58  B : 0.09  
Am	C : 0.24  E : 0.19  G : 0.10  A : 0.24  B : 0.24  
F	C : 0.30  D : 0.03  F : 0.40  A : 0.27  
Am	C : 0.23  E : 0.23  G : 0.04  A : 0.23  B : 0.27  
Am	C : 0.09  E : 0.09  G : 0.16  A : 0.58  B : 0.09  
Am	C : 0.24  E : 0.19  G : 0.10  A : 0.24  B : 0.24  
F	C : 0.30  D : 0.03  F : 0.40  A : 0.27  
Am	C : 0.23  E : 0.23  G : 0.04  A : 0.23  B : 0.27  
Am	C : 0.25  E : 0.25  A : 0.25  B : 0.25  
Am	C : 0.24  E : 0.19  G : 0.10  A : 0.24  B : 0.23  
F	C : 